# Introduction

In this notebook, at firdt, a table in a Wikipedia webpage is scraped, then the data are wrangled, cleaned and then rend into a pd dataframe.

Then the venues of different neighbors of Toronto are searched and segmented.

A clustering of neighbors will be done according to the most frequented venue categories.

# Part 1

In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

In [2]:
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

import requests
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [3]:
!pip install beautifulsoup4

In [4]:
!pip install lxml
!pip install html5lib

In [5]:
from bs4 import BeautifulSoup

In [6]:
import csv

source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')

csv_file=open('Toronto_neighbors.csv','w')
csv_writer=csv.writer(csv_file)
csv_writer.writerow(['Postal Code','Borough','Neighborhood'])

table=soup.find('table',class_='wikitable')

for onerow in table.find_all('tr'):
    try:
        element=onerow.find_all('td')
        postal_code=element[0].text.split('\n')
        postal_code=postal_code[0]
        print(postal_code)
        
        borough=element[1].text.split('\n')
        borough=borough[0]
        print(borough)
        
        neighborhood=element[2].text.split('\n')
        neighborhood=neighborhood[0]
        print(neighborhood)
        
        csv_writer.writerow([postal_code,borough,neighborhood])
    except:
        pass
    
csv_file.close()

M1A
Not assigned

M2A
Not assigned

M3A
North York
Parkwoods
M4A
North York
Victoria Village
M5A
Downtown Toronto
Regent Park / Harbourfront
M6A
North York
Lawrence Manor / Lawrence Heights
M7A
Downtown Toronto
Queen's Park / Ontario Provincial Government
M8A
Not assigned

M9A
Etobicoke
Islington Avenue
M1B
Scarborough
Malvern / Rouge
M2B
Not assigned

M3B
North York
Don Mills
M4B
East York
Parkview Hill / Woodbine Gardens
M5B
Downtown Toronto
Garden District, Ryerson
M6B
North York
Glencairn
M7B
Not assigned

M8B
Not assigned

M9B
Etobicoke
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
M1C
Scarborough
Rouge Hill / Port Union / Highland Creek
M2C
Not assigned

M3C
North York
Don Mills
M4C
East York
Woodbine Heights
M5C
Downtown Toronto
St. James Town
M6C
York
Humewood-Cedarvale
M7C
Not assigned

M8C
Not assigned

M9C
Etobicoke
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
M1E
Scarborough
Guildwood / Morningside / West Hill
M2E
Not assigne

In [7]:
# get the csv file scraped from wikipedia
df=pd.read_csv('Toronto_neighbors.csv')

# drop the rows with not assigned borough and reset the index
df=df[~df['Borough'].isin(['Not assigned'])]
df.reset_index(drop=True,inplace=True)

#verified by the method below, there no one postcode with 2 neighborhoods
#dfg=df.groupby('Postal code').count()
#dfg

print(df.shape)
df.head()

(103, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [8]:
df.set_index('Postal Code',inplace=True)
df.head()

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Regent Park / Harbourfront
M6A,North York,Lawrence Manor / Lawrence Heights
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


# Part 2

In [9]:
!pip install geocoder
import geocoder # import geocoder

     |████████████████████████████████| 102kB 8.4MB/s ta 0:00:011


In [10]:
file_path='http://cocl.us/Geospatial_data'

df_geo=pd.read_csv(file_path)
df_geo.set_index('Postal Code',inplace=True)
df_geo.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [11]:
df_latlng = pd.concat([df,df_geo],axis=1,join='inner')
print(df_latlng.shape)
df_latlng.head()

(103, 4)


,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [12]:
df_latlng.reset_index(inplace=True)
df_latlng.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


# Part 3

In [13]:
dfg=df_latlng.groupby('Borough').count()
dfg

,Postal Code,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,19,19,19,19
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Scarborough,17,17,17,17
West Toronto,6,6,6,6


In [14]:
df_toronto=df_latlng[df_latlng.Borough.isin(['Central Toronto','Downtown Toronto','East Toronto','West Toronto'])]
print(df_toronto.shape)
df_toronto.head()

(39, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [15]:
df_toronto.reset_index(drop=True,inplace=True)
df_toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [16]:
print(df_toronto.shape)

(39, 5)


In [17]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [23]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: | ^C
failed

CondaError: KeyboardInterrupt



ModuleNotFoundError: No module named 'folium'

In [ ]:
# map Toronto city
map_toronto=folium.Map(location=[latitude,longitude],zoom_start=12)

In [ ]:
# map different borough in Toronto city (just Borough with 'Toronto')

for lat, lng, label in zip(df_toronto['Latitude'],df_toronto['Longitude'],df_toronto['Neighborhood']):
    label=folium.Popup('label',parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        pupup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)

map_toronto
    


In [ ]:
# Define my credential and version of Foursquare
CLIENT_ID = 'GW1IMYKXGKDXQ02S5GN4AG2CJZZPFTKJKC5EMPXPAUT3JGIO' 
CLIENT_SECRET = 'AAKYS1EEKG4CIALKBQ2WLL01LYYQAZ0AOCR3J0MXCUVKY1IL' 
VERSION = '20200412' 

print('The credentails: CLIENT_ID {} and CLIENT_SECRET {}'.format(CLIENT_ID, CLIENT_SECRET))

In [ ]:
# create the function to explore the venues of each neighborhoods in df_toronto

LIMIT=50

def getNearbyVenues(names, latitudes, longitudes, radius=400):
    
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                'Neighborhood Latitude', 
                'Neighborhood Longitude', 
                'Venue', 
                'Venue Latitude', 
                'Venue Longitude', 
                'Venue Category']
    
    return(nearby_venues)

In [ ]:
# run the function with the data in df_toronto and create the dataframe toronto_venues


toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )
toronto_venues.head()

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

In [ ]:
toronto_venues.groupby('Neighborhood').count().head()

In [ ]:
# find the number of unique category in toronto_venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

In [ ]:
# analyse each neighborhood

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

print(toronto_onehot.shape)
toronto_onehot.head()

In [ ]:
# group by neighborhood and get the frequency of each category

toronto_grouped=toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

In [ ]:
toronto_grouped.shape

In [ ]:
# get each neighborhood the top 5 venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
# sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
# create a dataframe to display the top 10 venues of each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
# run the K-means to cluster the neighborhood in 5 cluster

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# create a new dataframe containing the clusters and the top 10 venues of each neighborhood

neighborhoods_venues_sorted.insert(0, 'Cluster labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with df_toronto to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

In [ ]:
kmeans.labels_

In [ ]:
# visualize the clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
colors_list=[0,1,2,3,4]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, ncolor in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster labels'],colors_list):
    label = folium.Popup('Cluster' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[ncolor-1],
        fill=True,
        fill_color=rainbow[ncolor-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
# examine each cluster
toronto_merged.loc[toronto_merged['Cluster labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]